In [1]:
from piTask import general
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from itertools import combinations
from scrapper import flipkart_json_scrapper_with_all_specifications as fk_scrapper
import pandas as pd
pd.set_option('display.max_columns', None)
from pprint import pprint 
import json
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import numpy as np
def open_chrome_headless():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    return driver
def fns_scrap(driver,url,no_of_pages):
    fsns = []
    for i in range(1, no_of_pages + 1):
        final_url = url + f'&page={i}'
        print(final_url)
        driver.get(final_url)
        all_fsns = driver.find_elements(By.XPATH, '//*[@data-id]')
        for fsn in all_fsns:
            single_fsn = fsn.get_attribute('data-id')
            fsns.append(single_fsn)        
    return fsns


def is_integer(value):
    try:
        int(value)
        return True
    except ValueError:
        return False
input_sheet='1YzEFAUQN-It8vxZ1OaPgkw5ZixVImj17nI-vkAjNz-0'
input_sheet_name ='input_sheet'
input_df=general.read_sheet(input_sheet,input_sheet_name,1)

keywords=[i for i in input_df['keyword'].to_list() if len(i)>0]
all_urls=['https://www.flipkart.com/search?q='+keyword.replace(" ","%20") for keyword in keywords]
my_fsns=[i for i in input_df['my_fsn'].to_list() if len(i)>0]
attributes_to_filter=input_df['attribute_to_match'].to_list()
brand_lists=[i for i in input_df['brand_name'].to_list() if len(i)>0]
driver=open_chrome_headless()
final_fsn_list=[]
for url in all_urls:
    fsns=fns_scrap(driver,url,10)
    final_fsn_list.extend(fsns)
driver.quit()
final_fsn_list=list(set(final_fsn_list))
print(len(final_fsn_list))
competitor_data=fk_scrapper.scrape_all_fsns(final_fsn_list)
my_data=fk_scrapper.scrape_all_fsns(my_fsns)
competitor_data.drop_duplicates(subset='fsn',inplace=True)
competitor_data['final_selling_price'] = competitor_data['final_selling_price'].apply(lambda x: x if is_integer(x) else None)
# my_data.dropna(subset=['final_selling_price'],inplace=True)
# my_data['final_selling_price'] = my_data['final_selling_price'].apply(lambda x: x if is_integer(x) else None)
# competitor_data.dropna(subset=['final_selling_price'], inplace=True)
# competitor_data['final_selling_price'] = competitor_data['final_selling_price'].astype(int)
# my_data['final_selling_price'] = my_data['final_selling_price'].astype(int)
# for brand in brand_lists:
#     print(brand)
#     substring_length=len(brand)
#     competitor_data.loc[competitor_data['title'].str[:substring_length]==brand,'brand']=brand
# competitor_data.dropna(subset=['brand'],inplace=True)
my_data.dropna(subset=['final_selling_price'],inplace=True)
my_data['final_selling_price']=my_data['final_selling_price'].replace([np.inf,-np.inf],0).fillna(0)
my_data['final_selling_price'] = my_data['final_selling_price'].apply(lambda x: x if is_integer(x) else None)
competitor_data.dropna(subset=['final_selling_price'], inplace=True)
competitor_data['final_selling_price']=competitor_data['final_selling_price'].replace([np.inf,-np.inf],0).fillna(0)
competitor_data['final_selling_price'] = competitor_data['final_selling_price'].astype(int)
my_data=my_data.dropna(subset=['final_selling_price'])
# my_data['final_selling_price'] = my_data['final_selling_price'].astype(int)
# for brand in brand_lists:
#     print(brand)
#     substring_length=len(brand)
#     competitor_data.loc[competitor_data['title'].str[:substring_length]==brand,'brand']=brand
# competitor_data.dropna(subset=['brand'],inplace=True)
# my_fsns=my_data['fsn'].to_list()


d:\personal_git\data science\portfolio projects\Competition analysis\.venv\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


d:\personal_git\data science\portfolio projects\Competition analysis\key1.json
https://www.flipkart.com/search?q=earphone&page=1
https://www.flipkart.com/search?q=earphone&page=2
https://www.flipkart.com/search?q=earphone&page=3
https://www.flipkart.com/search?q=earphone&page=4
https://www.flipkart.com/search?q=earphone&page=5
https://www.flipkart.com/search?q=earphone&page=6
https://www.flipkart.com/search?q=earphone&page=7
https://www.flipkart.com/search?q=earphone&page=8
https://www.flipkart.com/search?q=earphone&page=9
https://www.flipkart.com/search?q=earphone&page=10
https://www.flipkart.com/search?q=earbuds&page=1
https://www.flipkart.com/search?q=earbuds&page=2
https://www.flipkart.com/search?q=earbuds&page=3
https://www.flipkart.com/search?q=earbuds&page=4
https://www.flipkart.com/search?q=earbuds&page=5
https://www.flipkart.com/search?q=earbuds&page=6
https://www.flipkart.com/search?q=earbuds&page=7
https://www.flipkart.com/search?q=earbuds&page=8
https://www.flipkart.com/sea

In [6]:
my_fsns=my_data['fsn'].to_list()
final_df=[]
for requested_fsn in my_fsns:

    requested_data=my_data[my_data['fsn']==requested_fsn]
    self_seller_name=requested_data['Seller Name'].values[0] 
    filtered_competitor_data=competitor_data[competitor_data['Seller Name'] != self_seller_name]
    for attribute in [i for i in attributes_to_filter if len(i) > 0]:
        requested_filter=requested_data['all_details'].values[0].get(attribute)
        filtered_competitor_data=competitor_data[competitor_data['all_details'].apply(lambda x: x.get(attribute)) == requested_filter]
    
    
    for index,row in input_df.iterrows():
        if row['ranged_metric'] !="":
            ranged_metric=row['ranged_metric']
            ranged_value=row['range']
            # print(ranged_metric,ranged_value)
            
            if ranged_metric == 'final_selling_price':
                requested_price=requested_data['final_selling_price'].values[0]
                filtered_competitor_data=filtered_competitor_data[(filtered_competitor_data['final_selling_price']>requested_price*(1-ranged_value)) & (filtered_competitor_data['final_selling_price']<requested_price*(1+ranged_value))]

            else:
                filtered_competitor_data[ranged_metric]=filtered_competitor_data['all_details'].apply(lambda x: x.get(ranged_metric))
                filtered_competitor_data[ranged_metric]=filtered_competitor_data[ranged_metric].str.extract(r'(\d+\.?\d*)').astype(float).fillna(0)
                requested_range=requested_data['all_details'].apply(lambda x: x.get(ranged_metric)).str.extract(r'(\d+\.?\d*)').astype(float).fillna(0)
                requested_range=requested_range.iloc[0].values[0]

                filtered_competitor_data=filtered_competitor_data[(filtered_competitor_data[ranged_metric]>requested_range*(1-ranged_value)) & (filtered_competitor_data[ranged_metric]<requested_range*(1+ranged_value))]
    first_title = requested_data['title'][:1].values[0]
    first_description = requested_data['description'][:1].values[0]
    first_details = requested_data['all_details'][:1].values[0]

    if len(filtered_competitor_data) > 0:
        for index,row in filtered_competitor_data.iterrows():

            second_title=row['title']
            second_details=" ".join([f"{k}: {v}" for k, v in row['all_details'].items()])
            second_description=row['description']
            filtered_competitor_data.loc[index,'all_details_score']=fuzz.token_sort_ratio(first_details,second_details)
            filtered_competitor_data.loc[index,'title_score']=fuzz.partial_ratio(first_title,second_title)
            filtered_competitor_data.loc[index,'description_score']=fuzz.partial_ratio(first_description,second_description)
        top_10_competitors=filtered_competitor_data.sort_values(by=['title_score','description_score','all_details_score'],ascending=False).head(5)
        top_10_competitors['our_fsn']=requested_fsn
        our_fsn=top_10_competitors['our_fsn']
        top_10_competitors.insert(0,'requested_fsn',our_fsn)
        url_competitor='https://www.flipkart.com/blaupunkt-bh41-wireless-headphones-long-playtimei-40mm-drivers-foldable-built-mic-bluetooth-headset/p/itm73e3b649da013?pid='+top_10_competitors['fsn']
        top_10_competitors.insert(1,'url_competitor',url_competitor)
        final_df.append(top_10_competitors)
    else:
        print(requested_fsn)

all_competitor_data=pd.concat(final_df)
len(all_competitor_data.columns)
df_to_send=pd.concat([all_competitor_data,my_data])
general.add_sheet_name('1YzEFAUQN-It8vxZ1OaPgkw5ZixVImj17nI-vkAjNz-0','competitors',1)
general.print_sheet(1,df_to_send,'competitors','1YzEFAUQN-It8vxZ1OaPgkw5ZixVImj17nI-vkAjNz-0',1,3)

SPPHYG2UERHCXHPK
SPPGHW78FPQFSJCK


In [2]:
competitor_data.columns

Index(['fsn', 'title', 'rating', 'ratings_count', 'reviews_count',
       'special_price', 'final_selling_price', 'mrp', 'flipkart_assured',
       'Seller Name', 'highlights', 'paymentOptions', 'description',
       'productDescription', 'specifications', 'fk_other_seller_info',
       'reviews', 'productImagesCount', 'productVideosCount', 'image_link',
       'all_specs'],
      dtype='object')

In [5]:
competitor_data['brand']=competitor_data['title'].str.split(" ").str[0]

In [12]:
brand_level_data=competitor_data.copy()

In [33]:
for index, row in brand_level_data.iterrows():
    # Get the dictionary from the 'all_specs' column
    specs = row['all_specs']
    # Iterate over each key-value pair in the dictionary
    for key, value in specs.items():
        # If the column does not exist, create it with default None values
        if key not in brand_level_data.columns:
            brand_level_data[key] = None
        # Set the value for the current row in the new column
        brand_level_data.at[index, key] = value

In [56]:
most_important_feature={}
feature_name=[]
feature_count=[]
for column in brand_level_data.columns:
    feature_name.append(column)
    feature_count.append(brand_level_data[column].count())


In [57]:
most_important_feature['feature_name']=feature_name
most_important_feature['feature_count']=feature_count

In [59]:
important_features=pd.DataFrame(most_important_feature)

In [60]:
important_features.sort_values(by='feature_count',ascending=False)

,feature_name,feature_count
0,fsn,1090
1,title,1090
2,rating,1090
3,ratings_count,1090
4,reviews_count,1090
...,...,...
102,Transmitter Power Requirement,25
104,Weight Without Battery,13
103,Transmitter Battery,13
100,Weight Without Cord,10


In [64]:
important_features['feautre_present_in_%_of_products']=important_features['feature_count']*100/important_features['feature_count'].max()

In [99]:
filtered_important_features=important_features[important_features['feautre_present_in_%_of_products'] >=10]['feature_name'].to_list()

In [102]:
final_scrapped_data=brand_level_data[filtered_important_features]